<a href="https://colab.research.google.com/github/SIVASKANDAN-P/PYTHON-CODE-HINT/blob/main/model_finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets
!pip install -U datasets
!pip install bert_score
!pip install peft
!pip install accelerate
!pip install bitsandbytes
!pip install trl
!pip install datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 19.0 MB/s eta 0:00:00
   

In [3]:
#importing necessary libraries
from datasets import load_dataset, load_dataset_builder
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer
from bert_score import score
from datetime import datetime
import os
import sys

import torch
from peft import (
    LoraConfig,
    get_peft_model,
    get_peft_model_state_dict,
#    prepare_model_for_int8_training,
    prepare_model_for_kbit_training,
    set_peft_model_state_dict,
    PeftModel
)
from transformers import BitsAndBytesConfig
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForSeq2Seq
from transformers import T5Tokenizer, T5ForConditionalGeneration
from trl import SFTTrainer
from datasets import Dataset

# Connecting to gdrive
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [4]:
def prompt_instruction_format(sample):
  return f"""### Instruction:
    Use the Task below and the Input given to write the Response:

    ### Task:
    Break down the give coding problem into its logical steps

    ### Input:
    {sample['instruction']}

    ### Response:
    {sample['response']}
    """

def tokenize_function(sample):
    return tokenizer(sample['instruction'], padding="max_length", truncation=True)

def compute_metrics(eval_pred):
    """Computes and returns a dictionary of metrics (e.g., accuracy, f1)."""

    # Unpack predictions and labels
    predictions, labels = eval_pred

    # Define and calculate desired metrics using 'evaluate' library
    metric = evaluate.load("rouge")
    # result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = metric.compute(predictions=predictions, references=labels)

    return result

In [5]:
model1 = "google/flan-t5-large"
model2 = "google/flan-t5-small"
model3 = "google/flan-t5-base"
model4 = "google/flan-t5-xl"
model5 = "google/flan-t5-xxl"
model6 = "codellama/CodeLlama-7b-hf"

In [16]:
base_model = model6

In [17]:
try:
  tokenizer = AutoTokenizer.from_pretrained(base_model)
except:
  tokenizer = T5Tokenizer.from_pretrained(base_model)

tokenizer_config.json:   0%|          | 0.00/749 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

In [18]:
dataset = pd.read_csv("/content/drive/MyDrive/dataset/data300.csv")
dataset = dataset[['instruction','response']]
train_dataset, test_dataset = train_test_split(dataset, test_size=0.2)
# train_dataset_tokenised Pandas DataFrame to hugging face dataset
train_dataset_hf = Dataset.from_pandas(train_dataset)
train_dataset_tokenized = train_dataset_hf.map(tokenize_function, batched=True)
# train_dataset_tokenised Pandas DataFrame to hugging face dataset
test_dataset_hf = Dataset.from_pandas(test_dataset)
test_dataset_tokenized = test_dataset_hf.map(tokenize_function, batched=True)

Map:   0%|          | 0/18706 [00:00<?, ? examples/s]

Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no padding.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/4677 [00:00<?, ? examples/s]

In [19]:
quantization_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_quant_type="nf4")

In [20]:
config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules="all-linear",  # To apply LoRA to all the linear layers
    lora_dropout=0.05,
    bias="none",                  #since for other when adopter is removed model will not perform the same
    task_type="CAUSAL_LM"         #since it is text generation task
)

#model = T5ForConditionalGeneration.from_pretrained(
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    #use_gradient_checkpointing=True
    quantization_config=quantization_config
)
peft_model = get_peft_model(model, config)


config.json:   0%|          | 0.00/637 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now default to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [ ]:
# model = T5ForConditionalGeneration.from_pretrained(
#     base_model,
#     #use_gradient_checkpointing=True
#     quantization_config=quantization_config
# )

In [21]:
# Define training arguments
training_args = TrainingArguments(
    output_dir='/content/drive/MyDrive/model',
    per_device_train_batch_size=1,
    gradient_accumulation_steps=2,
    num_train_epochs=1,
    learning_rate=1e-4,
    fp16=True,
    logging_dir="/content/drive/MyDrive/model/log",
    evaluation_strategy="steps",
    save_strategy="epoch",
    eval_steps=4000,
    save_steps=1000,
    per_device_eval_batch_size=1,  # Reduce eval batch size to reduce memory during evaluation
)


# Create the Trainer instance with SFTTrainer
trainer = SFTTrainer(
    model=peft_model,
    args=training_args,
    train_dataset=train_dataset_tokenized, # Your tokenized dataset
    compute_metrics = compute_metrics,
    eval_dataset=test_dataset_tokenized            # Optional validation dataset
    # data_collator=data_collator, # Optional data collator
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:309: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:403: UserWarning: You passed a processing_class with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `processing_class.padding_side = 'right'` to 

In [ ]:
trainer.train()

Step,Training Loss,Validation Loss


In [ ]:

# Load the model and tokenizer
loaded_tokenizer = T5Tokenizer.from_pretrained('/content/drive/MyDrive/model/google/flan-t5-base_token_20241106_114401')
loaded_model = T5ForConditionalGeneration.from_pretrained('/content/drive/MyDrive/model/google/flan-t5-base_20241106_114401')

In [ ]:
import datetime
# Get the current timestamp
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
print(timestamp)
# Save the model to a specified directory
trainer.save_model(os.path.join('/content/drive/MyDrive/model', f"{base_model}_{timestamp}"))

# Optionally, save the tokenizer as well
trainer.tokenizer.save_pretrained(os.path.join('/content/drive/MyDrive/model', f"{base_model}_token_{timestamp}"))

In [ ]:
config = model.config  # Assuming 'model' is your loaded model
max_seq_length = config.max_position_embeddings
print(f"Maximum sequence length: {max_seq_length}")